In [229]:
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import re
import numpy as np
response = get('https://philadelphia.craigslist.org/search/apa?') 
html_soup = BeautifulSoup(response.text, 'html.parser')
posts = html_soup.find_all('li', class_ = 'result-row')

In [225]:
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_= 'totalcount').text)

In [205]:
from time import sleep
import re
from random import randint
from warnings import warn
from time import time
from IPython.core.display import clear_output
import numpy as np

#find the total number of posts to find the limit of the pagination
results_num = html_soup.find('div', class_= 'search-legend')
results_total = int(results_num.find('span', class_= 'totalcount').text) 

pages = np.arange(0, results_total+1, 120)

iterations = 0

post_timing = [ ]
post_hoods = []
post_title_texts = []
bedroom_counts = []
sqfts = []
post_links = []
post_prices = []

for page in pages:
    
    
    response = get("https://philadelphia.craigslist.org/search/apa?" 
                   + "s=" 
                   + str(page) 
                   + "&hasPic=1"
                   + "&availabilityMode=0")

    sleep(randint(1,5))
     
    
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))
        
    #define the html text
    page_html = BeautifulSoup(response.text, 'html.parser')
    
    #define the posts
    posts = page_html.find_all('li', class_= 'result-row')
        
    #extract data item-wise
    for post in posts:

        if post.find('span', class_ = 'result-hood') is not None:

            post_datetime = post.find('time', class_= 'result-date')['datetime']
            post_timing.append(post_datetime)

            #neighborhoods
            post_hood = post.find('span', class_= 'result-hood').text
            post_hoods.append(post_hood)

            #title text
            post_title = post.find('a', class_='result-title hdrlnk')
            post_title_text = post_title.text
            post_title_texts.append(post_title_text)

            #post link
            post_link = post_title['href']
            post_links.append(post_link)
            
            
            post_price= post.a.text.strip().replace("$", "").replace(",", "")
            if post_price == '':
                post_price == np.nan
                post_prices.append(post_price)
            else:
                post_prices.append(post_price)
            
            if post.find('span', class_ = 'housing') is not None:
                
                
                if 'ft2' in post.find('span', class_ = 'housing').text.split()[0]:
                    
                    
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                    
                    
                    sqft = int(post.find('span', class_ = 'housing').text.split()[0][:-3])
                    sqfts.append(sqft)
                    
                
                elif len(post.find('span', class_ = 'housing').text.split()) > 2:
                    
                    
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    
                    sqft = int(post.find('span', class_ = 'housing').text.split()[2][:-3])
                    sqfts.append(sqft)
                    
                
                elif len(post.find('span', class_ = 'housing').text.split()) == 2:
                    
                    
                    bedroom_count = post.find('span', class_ = 'housing').text.replace("br", "").split()[0]
                    bedroom_counts.append(bedroom_count)
                    
                    
                    sqft = np.nan
                    sqfts.append(sqft)                    
                
                else:
                    bedroom_count = np.nan
                    bedroom_counts.append(bedroom_count)
                
                    sqft = np.nan
                    sqfts.append(sqft)
                
   
            else:
                bedroom_count = np.nan
                bedroom_counts.append(bedroom_count)
                
                sqft = np.nan
                sqfts.append(sqft)

                
    iterations += 1
    print("Page " + str(iterations) + " scraped successfully!")

print("\n")

print("Scrape complete!")

Page 1 scraped successfully!
Page 2 scraped successfully!
Page 3 scraped successfully!
Page 4 scraped successfully!
Page 5 scraped successfully!
Page 6 scraped successfully!
Page 7 scraped successfully!
Page 8 scraped successfully!
Page 9 scraped successfully!
Page 10 scraped successfully!
Page 11 scraped successfully!
Page 12 scraped successfully!
Page 13 scraped successfully!
Page 14 scraped successfully!
Page 15 scraped successfully!
Page 16 scraped successfully!
Page 17 scraped successfully!
Page 18 scraped successfully!
Page 19 scraped successfully!
Page 20 scraped successfully!
Page 21 scraped successfully!
Page 22 scraped successfully!
Page 23 scraped successfully!
Page 24 scraped successfully!
Page 25 scraped successfully!
Page 26 scraped successfully!


Scrape complete!


In [207]:
df_scrape = np.column_stack((post_title_texts,post_prices, sqfts, post_links,bedroom_counts, post_hoods,post_timing))

In [208]:
philly_craiglist_housing = pd.DataFrame(df_scrape)


In [209]:
philly_craiglist_housing.shape

(2453, 7)

In [210]:
philly_craiglist_housing.columns = ['post_title_texts','post_prices', 'sqfts', 'post_links','bedroom_counts','post_hoods','post_timing']

In [211]:
philly_craiglist_housing['post_title_texts'].value_counts()

NEW CONSTRUCTION 3BR 3BA BI-LEVEL UNIT IN NORRIS SQUARE!!!                25
Great studio available now at 3412 Spring garden!                         18
1BR At Chestnut Park Apartments                                           17
THE SOCIETY HILL BUILDING IS A BEAUTIFUL MID-RISE COMMUNITY LOCATED IN    14
Spacious Apt./Parking "Great Location" Must See! Move-In Special!         14
                                                                          ..
Call for Details, Poolside Grill, Pets Welcome                             1
Close to Freeway, Close to Shopping, Tile Backsplash                       1
1737 South Street, Unit 2- Fantastic Apartment on South Street!            1
2 bed- Valet Dry Cleaning, Lap Pool, Designer flooring throughout          1
Act Now! Drexel U Luxury 3 bed 2 bath bi-level apt available in July!!     1
Name: post_title_texts, Length: 1807, dtype: int64

In [212]:
philly_craiglist_housing.to_csv('philly_craig_housing_list.csv')

In [250]:
df_philly = pd.read_csv('philly_craig_housing_list.csv')

In [215]:
philly_craiglist_housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2453 entries, 0 to 2452
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   post_title_texts  2453 non-null   object
 1   post_prices       2453 non-null   object
 2   sqfts             2453 non-null   object
 3   post_links        2453 non-null   object
 4   bedroom_counts    2453 non-null   object
 5   post_hoods        2453 non-null   object
 6   post_timing       2453 non-null   object
dtypes: object(7)
memory usage: 134.3+ KB


In [218]:
post_link= df_philly['post_links']

In [241]:
No_of_bathrooms = []
No_of_bedrooms = []
latitude = []
longitude = []
pet=[]
dog = []
laundry =[]
parking =[]
floor = []
Apartment_type = []
address = []
price = []
smoking = []
over_all = []


for i in post_link:
    indi_response = get(i)
     
    #define the html text
    page_html = BeautifulSoup(indi_response.text, 'html.parser')

    #define the posts
    ind_posts = page_html.find_all('div', class_ = 'mapAndAttrs')
    for links in ind_posts:
        
        #the bedroom , bathroom and space
        if links.find('p', class_ = 'attrgroup') is not None:
            
            apartment_phys = links.find('p', class_= 'attrgroup')

        
        #bedroom -bathroom
            No_of_bedrooms.append(apartment_phys.span.text.split('/')[0])
            No_of_bathrooms.append(apartment_phys.span.text.split('/')[1])
        
        #latitude and longitude
        if links.find('div', class_ = "viewposting") is None:
            longitude.append(np.nan)
            latitude.append(np.nan)
        else:
            long_lat = links.find('div', class_ = "viewposting")
            if long_lat['data-longitude']=='':
                long_lat['data-longitude'] = np.nan
                longitude.append(np.nan)
            else:
                longitude.append(float(long_lat['data-longitude']))
            if long_lat['data-latitude']=='':
                long_lat['data-latitude'] = np.nan
                latitude.append(np.nan)
            else:
                latitude.append(float(long_lat['data-latitude']))
        

        
        if links.find_all('p', class_ = 'attrgroup')[1] is not None:
            t = links.find_all('p', class_ = 'attrgroup')[1]
            
            span=t.text.split('\n')
            
            if 'cats are OK - purrr' in span and 'dogs are OK - wooof' in span:
                pet.append('both pets')
            elif 'dogs are OK - wooof'in span:
                pet.append('dogs')
            elif 'cats are OK - purrr'in span:
                pet.append('cats')
            else:
                pet.append('no pet')
                
        
            #laundry
            if 'laundry in bldg' in span:
                laundry.append('on site')
            elif 'w/d in unit' in span:
                laundry.append('in unit')
            elif 'laundry on site' in span:
                laundry.append('on site')
            else:
                laundry.append('no laundry')
                
            #house
            if 'cottage/cabin'in span:
                Apartment_type.append('single')
            elif 'duplex' in span:
                Apartment_type.append('double')
            elif 'house' in span:
                Apartment_type.append('single')
            elif 'in-law' in span:
                Apartment_type.append('double')
            elif 'townhouse' in span:
                Apartment_type.append('single')
            else:
                Apartment_type.append('multi')
            
            #floor
            if 'flooring: carpet' in span:
                floor.append('carpet')
            elif 'flooring: wood' in span:
                floor.append('wood')
            else:
                floor.append(np.nan)
                

        
            #Parking
            if 'carport' in span:
                parking.append('protected')
            elif 'off-street parking' in span:
                parking.append('off-street')
            elif 'detached garage' in span:
                parking.append('protected')
            elif 'attached garage' in span:
                parking.append('proctected')
            elif 'valet parking' in span:
                parking.append('valet')
            else:
                parking.append('no')
            
            #smoking
            if 'smoking' in span:
                smoking.append('smoking')
            elif 'no smoking' in span:
                smoking.append('no smoking')
            else:
                smoking.append(np.nan)
            
            #overall
            over_all.append(span)
            
            

In [243]:
#create the numpy array
df_np = np.stack((No_of_bathrooms,No_of_bedrooms, latitude, longitude, pet, laundry, parking, floor, Apartment_type, smoking), axis=-1)

In [245]:
col_names = 'No_of_bathrooms', 'No_of_bedrooms', 'latitude', 'longitude', 'pet', 'laundry', 'parking', 'floor', 'Apartment_type', 'smoking'

In [251]:
#create the dataframe from numpy arrays
df = pd.DataFrame(data=df_np, columns= col_names)

In [252]:
df.head(10)

,No_of_bathrooms,No_of_bedrooms,latitude,longitude,pet,laundry,parking,floor,Apartment_type,smoking
0,2Ba,2BR,39.969515,-75.139578,both pets,in unit,no,nan,multi,nan
1,1Ba,1BR,39.9207,-75.1595,no pet,in unit,off-street,wood,multi,nan
2,1Ba,1BR,39.9597,-75.2024,no pet,in unit,no,nan,multi,nan
3,1Ba,1BR,39.9483,-75.1433,no pet,in unit,no,wood,multi,nan
4,1Ba,1BR,39.972642,-75.179142,both pets,no laundry,no,wood,multi,no smoking
5,2Ba,2BR,39.954424,-75.163215,both pets,in unit,no,nan,multi,no smoking
6,1Ba,2BR,39.987611,-75.818503,both pets,on site,off-street,nan,multi,nan
7,1Ba,1BR,39.95618,-75.194152,both pets,on site,protected,nan,multi,nan
8,1Ba,0BR,39.954424,-75.163215,both pets,in unit,no,nan,multi,no smoking
9,1Ba,2BR,39.944711,-75.158214,cats,on site,no,nan,multi,nan


In [253]:
df.to_csv('df_crailist_housing_web_scrap_philly_indi_listing.csv')

In [254]:
#combine the two dataframe
df_combo = pd.concat([df_philly, df], axis=1)

In [255]:
df_combo

,Unnamed: 0,post_title_texts,post_prices,sqfts,post_links,bedroom_counts,post_hoods,post_timing,No_of_bathrooms,No_of_bedrooms,latitude,longitude,pet,laundry,parking,floor,Apartment_type,smoking
0,0,👉 Experience The Virtual Tours at The Piazza,3045,1002.0,https://philadelphia.craigslist.org/apa/d/phil...,2.0,(1050 N Hancock St - Northern Liberties),2021-02-18 17:22,2Ba,2BR,39.969515,-75.139578,both pets,in unit,no,nan,multi,nan
1,1,Spacious 1 bedroom apartment,1000,NaN,https://philadelphia.craigslist.org/apa/d/phil...,1.0,(South Philadelphia),2021-02-18 17:19,1Ba,1BR,39.9207,-75.1595,no pet,in unit,off-street,wood,multi,nan
2,2,"Stainless GE Appliances, Wood Floors, Rooftop ...",2487,NaN,https://philadelphia.craigslist.org/apa/d/phil...,1.0,(Philadelphia),2021-02-18 17:19,1Ba,1BR,39.9597,-75.2024,no pet,in unit,no,nan,multi,nan
3,3,"Loft Apartment with Maple Hardwood floors, Exp...",1690,1100.0,https://philadelphia.craigslist.org/apa/d/phil...,1.0,(Old City),2021-02-18 17:18,1Ba,1BR,39.9483,-75.1433,no pet,in unit,no,wood,multi,nan
4,4,"PRICE IMPROVEMENT! Spacious 1BR w/ H/W Flrs, D...",1100,NaN,https://philadelphia.craigslist.org/apa/d/phil...,1.0,"(2636 Poplar St #2, Fairmount)",2021-02-18 17:16,1Ba,1BR,39.972642,-75.179142,both pets,no laundry,no,wood,multi,no smoking
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,2448,"1303 S. Broad Large One Bedroom ,Marble Bath",1300,NaN,https://philadelphia.craigslist.org/apa/d/1303...,1.0,(South Philadelphia),2021-02-16 14:27,1Ba,1BR,39.9362,-75.1563,both pets,in unit,no,wood,multi,no smoking
2449,2449,Love Where You Live!!,2177,780.0,https://philadelphia.craigslist.org/apa/d/phil...,1.0,(Philadelphia),2021-02-16 14:26,1Ba,1BR,39.958523,-75.165587,both pets,in unit,proctected,nan,multi,no smoking
2450,2450,"842 sqft 1 bd, 1 ba ready to lease! Come on ov...",1389,842.0,https://philadelphia.craigslist.org/apa/d/west...,1.0,(1515 Manley Rd),2021-02-16 14:26,1Ba,1BR,39.967056,-75.529129,both pets,in unit,off-street,nan,multi,no smoking
2451,2451,🔥🔥Check out HamCo… Things are HEATING UP🔥🔥,5895,1610.0,https://philadelphia.craigslist.org/apa/d/phil...,5.0,(101 S. 39th Street - University City),2021-02-16 14:25,2Ba,5BR,39.955341,-75.19994,both pets,in unit,no,nan,multi,nan


In [256]:
df_combo.to_csv('web_scrap_craiglist_data_philly_final.csv')

In [258]:
len(df_combo.post_title_texts.value_counts())

1807